# Power Spectra Part 2 - Individual Map splits and Nawrapper 

This notebook can be used to generate an example power spectrum using [nawrapper](https://github.com/xzackli/nawrapper), a [Namaster](https://github.com/LSSTDESC/NaMaster) wrapper.  For this section we use the maps that are presented in [Aiola et al 2020](https://phy-act1.princeton.edu/public/saiola/act_dr4_A20.pdf) which are the maps of individual patches. These maps include multiple splits along with the other products needed to generate power spectra from them. In this example we walk through how to use Nawrapper to generate a spectrum of ACT's D56 region. This follows the same procedure as used in [Choi et al 2020](https://phy-act1.princeton.edu/public/saiola/act_dr4_C20.pdf).

It's worth noting that this notebook relies on computations that are reasonably memory intensive and so, if you're using this with a docker container, it is worth checking that the memory settings have been increased from their default settings.  In general we actually perform computations like this on clusters using scripts, opposed to in notebook format on individual laptops.  With that in mind please use this code as an example and consider adapting it for use in a script if you would like to use this approach in your research.

This code relies on maps dicussed in [Aiola et al 2020](https://phy-act1.princeton.edu/public/saiola/act_dr4_A20.pdf) and techniques from [Choi et al 2020](https://phy-act1.princeton.edu/public/saiola/act_dr4_C20.pdf), as such we ask that you reference those orginal works when using these data products or this code.


Notebook Info | Value
---|---
Dependencies | `numpy`, `scipy`, `healpy`, `matplotlib`, `pixell`, `nawrapper`
Data products | "BIN_ACTPOL_50_4_SC_low_ell"
| "act_dr4.01_s15_pa1_f150_nohwp_night_beam_tform_jitter_D56_cmbspec.txt" 
| "act_dr4.01_mask_s13s16_0.015mJy_5.0arcmin.fits"
| "act_dr4.01_D56_apo_w0.fits" 
| "act_dr4.01_s15_D56_pa1_f150_nohwp_night_3pass_4way_coadd_ivar.fits" 
| act_dr4.01_s15_D56_pa1_f150_nohwp_night_3pass_4way_set{0-3}_map_srcfree.fits
| act_dr4.01_s15_D56_pa1_f150_nohwp_night_3pass_4way_set{0-3}_srcs.fits.fits
| camb_theory.dat
| deep56_TT_spectrum.txt
| deep56_kfilter_tfunc_sim.txt
Products available to read from disk| mcm/mcm.json and mcm/w00.bin
this notebook can also generate these products| TT_cross_spectra.txt
| cross_cov_TTTT.txt
| auto_cov_TTTT.txt
Memory usage | High (lower memory options available)
Contributors |  Emilie Storer, Zack Li, Maya Mallaby-Kay, Zach Atkins

To return to the main notebook or to view a different section use the links in the table of contents.

## Table of contents

 --- 
> [Section 1](Section_1_Introduction.ipynb): Overview and Introduction

> [Section 2](Section_2_coadded_maps.ipynb): Coadded maps from [Naess et al (2020)](https://phy-act1.princeton.edu/public/saiola/act_dr4_N20.pdf) and an Introduction to Pixell

> [Section 3](Section_3_HEALPix_CAR_Conversions.ipynb):  Demonstration of methods to convert maps between HEALPix format and the CAR format.

> [Section 4](Section_4_visualize_objects.ipynb):  The DR5 Cluster Catalog from [Hilton et al (2020)](https://arxiv.org/abs/2009.11043) and Visualizing Objects in the Maps

> [Section 5](Section_5_Lensing_maps.ipynb):  Introduction to the Lensing maps from [Darwish et al (2020)](https://arxiv.org/pdf/2004.01139.pdf)

> [Section 6](Section_6_Component_separated_maps.ipynb):  Guide to the use of the Component Separated maps from [Madhavacheril et al (2019)](https://arxiv.org/abs/1911.05717)

> [Section 7](Section_7_power_spectra_part_1.ipynb): Power Spectra Part 1 - Harmonic Analysis using CAR maps [Naess et al (2020)](https://phy-act1.princeton.edu/public/saiola/act_dr4_N20.pdf)

> [Section 8](Section_8_power_spectra_part_2.ipynb) Power Spectra Part 2 - Power Spectrum example using Nawrapper [Aiola et al (2020)](https://phy-act1.princeton.edu/public/saiola/act_dr4_A20.pdf) and [Choi et al (2020)](https://phy-act1.princeton.edu/public/saiola/act_dr4_C20.pdf) 

> [Section 9](Section_9_Noise_sims.ipynb): Generating Noise Simulations 

> [Section 10](Section_10_E_B_maps.ipynb):  Look at how to generate E and B maps from I, Q, U maps. 

> [Section 11](Section_11_ACT_likelihood.ipynb): Using the ACT Likelihood code

> [Section 12](Section_12_delensing_param_shifts.ipynb):  Delensing the CMB powerspectrum and comparing the delensed and lensed parameters from [Han et al. (2020)](https://arxiv.org/pdf/2007.14405.pdf)
 
---

## Accessing the available map splits

The season 13 through 16 maps cover ~ 17,000 sq degrees on the sky and we present both 98 GHz and 150 GHz maps. They include data used to make the maps presented in [Naess et al (2014)](https://doi.org/10.1088/1475-7516/2014/10/007) and [Louis et al (2017)](https://doi.org/10.1088/1475-7516/2017/06/031) along with sumplemental observations from 2015 and 2016. While we will only use the data from the D56 patch in this particular notebook, the rest of the patch, array, season and frequncy combinations are detailed in Table 1 and in [Aiola et al 2020](https://phy-act1.princeton.edu/public/saiola/act_dr4_A20.pdf).

---
Table 1: Summary of patch sizes and white-noise levels
---



Season | s13|s14|s15|s16
---|:-:|:-:|:-:|:-:
**Array and Frequency** | **PA1$_{150}$** | **PA1$_{150}$ $|$ PA2$_{150}$**|  **PA1$_{150}$** $|$ **PA2$_{150}$ $|$ PA3$_{98}$ $|$ PA3$_{150}$**| **PA2 $|$ PA3$_{98}$ $|$ PA3$_{150}$**
**D1 (131 sq deg)**| 18.6 | | |
**D5 (157 sq deg)** | 16.4 
**D6 (135 sq deg)**| 12.6|
**D56 (834 sg deg)**| | 32.3 $|$ 20.5 | 33.3 $|$ 21.9 $|$ 18.4 $|$ 28.6|
**D8 (248 sq deg)**| | | 42.0 $|$ 21.5 $|$ 20.0 $|$ 29.0|
**BN (3157 sq deg)**| | |76.8 $|$ 41.3 $|$ 33.9 $|$ 49.1
**AA (17044 sq deg)** | | || 72.9 $|$ 78.7 $|$ 118.5

White noise levels are given in units of $\mu$K-arcmin.  Table reproduced from [Aiola et al](https://phy-act1.princeton.edu/public/saiola/act_dr4_A20.pdf) 

---

For each of the patch, season, array and frequency combination we include 4 splits (with the exception of patch AA which includes 2) as well as one map-based co-add resulting in 94 data units.  

Each of these units contains 4 fits files:
 * Source-free maps:  These contain I (or T), Q and U Stokes components and are in units of $\mu K_{CMB}$ 
 * Source maps:  These contain the point-source signal in units of $\mu K_{CMB} $ .  The simple sum of the source and source-free maps will give a map of the observed sky.
 * ivar (inverse-variance) maps: Roughly proportional to a traditinoal hit count map but includes the variance contribution from detector noise and so is better suited for pixel weighting in map-based co-addition.
 * Cross-linking maps: T, Q, U components that are used to describe the pixel cross-linking of a dataset.  

---
Maps are named according to the convention:
> act_dr4.01_season\*\_patch\*\_array\*\_freq\*\_nohwp_night_3pass_set\*\_map_suffix\*

* Season: s13, s14, s15, s16
* patch: D1, D5, D6, D56, D8, BN, AA
* array: PA1, PA2, PA3
* freq: f090, f150
* set: 2way_set0, 2way_set1 or 4way_set(0 - 4)
* map_suffix: map_srcfree.fits, srcs.fits, xlink.fits, ivar.fits

Once the splits of interest have been chosen they can be read in and viewed using Pixell's enmap as shown in the following example.

For this example we use the D56 patch at 150 GHz.

### Set up the notebook and import modules

As usual we begin by reading in the first notebook to set up our path and import dependencies.  We will also take a moment to import nawrapper and set up the plotting function we've been using throughout.

In [ ]:
# %run Section_1_Introduction.ipynb
from pixell import enplot, enmap
import nawrapper as nw

path = '/mnt/c/Users/gdzhao/projects/unwise_sz/act_DR4_DR5/'
# Redefine our plotting function from section 2
def eshow(x,**kwargs): 
    ''' Define a function to help us plot the maps neatly '''
    plots = enplot.get_plots(x, **kwargs)
    enplot.show(plots, method = "ipython")

### Loading the relevant maps

Before discussing our map-domain data, we first load a set of five auxilliary data files: the ell bins used for ACT power spectra, the beam file, the point source mask, the apodized patch footprint, and the inverse-variance map.

In [ ]:
bin_file = "BIN_ACTPOL_50_4_SC_low_ell"
beam_file = "act_dr4.01_s15_pa1_f150_nohwp_night_beam_tform_jitter_D56_cmbspec.txt" 
psmask_file = "act_dr4.01_mask_s13s16_0.015mJy_5.0arcmin.fits"
footprint_file = "act_dr4.01_D56_apo_w0.fits" 
ivar_file = "act_dr4.01_s15_D56_pa1_f150_nohwp_night_3pass_4way_coadd_ivar.fits" 

In [ ]:
# normalize the beam
beam = nw.read_beam(path + beam_file)
beam /= beam[0]

We extract the shape and wcs of the footprint, and then ensure that all the maps we use to compute the power spectrum have this same shape and wcs.

In [ ]:
# read in the footprint
footprint = enmap.read_map(path + footprint_file)
shape,wcs = footprint.shape, footprint.wcs

# read in the point source mask, make sure it has the correct shape, and apodize
psmask = enmap.extract(enmap.read_map(path + psmask_file), shape, wcs)
psmask = nw.apod_C2(psmask, 18./60.)

# read in the coadd inverse variance map and make sure it has the correct shape
ivar = enmap.extract(enmap.read_map(path + ivar_file), shape, wcs)

mask = footprint*psmask*ivar

del ivar, footprint, psmask

In [ ]:
# We can plot the final mask quickly to look at it 

eshow(mask, **{"colorbar": True})

Our final mask is made up of the apodized footprint corresponding to the region of sky (D56) we want to study, an apodized point source mask, and the area is weighted by the inverse variance of the coadded map.

### Map Processing

Next, we load in the maps themselves. In this example, the same mask is used to speed up the spectra calculation, but in general each map will have a different mask.

As discussed earlier, these maps include 4 splits that are designed to make it possible to calculate multiple cross spectra of the same patch.  In order to use the four splits we need to ensure that they all have the same geometry and that we've correctly combined the source and source free maps for each patch.  

In this next section we will go through and combine the maps and check that the geometries all match up.  We will also take a moment to apply the typical map preprocessing that is done in ACT map analysis.  In particular we correct for the anistropy of the pixel window and then we also apply filtering to the fourier modes such by removing the horizontal noise modes.  This is done with the `preprocess_fourier` function, which will also call `enmap.extract` if you pass it `shape` and `wcs` information, in order to ensure that all the maps have the same geometry.

Finally we will take a moment to address the computational difficulties with this particular process.  In general this notebook contains code that would normally be run using scripts that get submitted to clusters.  With that in mind we have provided the code you would need to run in order to do these calculations, but we have also provided users with the option to read precalculated data products in from disk instead of running the code themselves.  For that reason the cell below is set to run with 2 splits so that you can see how it works but we will later provide you with the neccesary data products that were generated using all 4 splits.  If you wish to run with all 4 splits at this point then simply change 'nsplits' back to 4.


In [ ]:
mapname_head = "act_dr4.01_s15_D56_pa1_f150_nohwp_night_3pass_4way_set"

# loop over splits and generate namap_list
# change this to 4 if you wish to work with all 4 splits 
# (note this will take much longer to run)
nsplits = 2   

namap_list = []
for i in range(nsplits):
    
    # read source-free map from disk and preprocess (i.e. k-space filter and pixwin)
    maps = enmap.read_map(path + f"{mapname_head}{i}_map_srcfree.fits")[0]
    maps = enmap.extract(maps,shape,wcs)
    map_I = nw.preprocess_fourier(maps, shape, wcs)
    del maps
    
    # add in the sources
    source_map = enmap.read_map(path + f"{mapname_head}{i}_srcs.fits")[0]
    source_map = enmap.extract(source_map,shape,wcs)
    map_I = map_I + source_map
    del source_map
    
    # create the namap_car, to bundle maps/masks/beams together
    split_namap = nw.namap_car(
        maps=map_I,
        masks=mask, 
        beams=beam)
    namap_list.append(split_namap)

The polarization components of the maps can also be included in the step above to compute the TE and EE spectra by specifying `map_Q` and `map_U`. This results in a longer computation time. For the polarization components a different beam and mask may be specified. 

### Computing the Mode Coupling Matrix


The next step is to compute our mode coupling matrix.  This is an important step for this process, but it can also be really slow to run (and for some laptops you may not be able to run it at all). With that in mind we have also provided you the option to load the matrix from disk instead of calculating it here.  Currently it will default to reading the matrix from disk, but if you'd like to run it your self feel free to do so.  

In [ ]:
# load the ell bins
bins = nw.read_bins(path + bin_file, is_Dell=True)

# # To calculate the mode coupling matrix yourself instead of loading it from disk simply uncomment this block
# mc = nw.mode_coupling(namap_list[0], namap_list[1], bins)

# Load the matrix from disk
mc = nw.mode_coupling(mcm_dir = path + "mcm")

This mode coupling matrix is used to account for the effects of the weight function, which includes the point source mask, inverse variance weighting etc. , when calculating the true full sky angular spectrum from the pseudo power spectrum given by our maps.  In the previous notebook we did this by simply weighting the power spectrum by the average of the mask which we mentioned was less than ideal.  Here we use the mode coupling matrix in order to more carefully account for the differences between the pseudo-Cls and the true Cls.

It's worth noting that the function above outputs a file called `w00` which refers to just the spin-0, spin-0 mode coupling matrix.  However, if you added the polarization maps in to `namap_list` you could also generate the `w02`, `w20`, and `w22` components which correspond to (spin-0, spin-2), (spin-2, spin-0) and (spin-2, spin-2) respectively.  

### Computing Spectra


Now that we have this matrix our next step is to apply it to each pair of namap objects in order to generate spectra. We will reuse the mode coupling object we computed, since all the masks are the same in this toy example. If you are using all 4 maps then this process wil provide 6 cross spectra that can be averaged to get a mean spectra.  We also take a moment to compute the standard error which we will use as quick error bars in the next step. Again, this step is computationally intensive, so we provide the data for you to load from disk (by default).  The data on disk was generated using all 4 splits. 

In [ ]:
TT_cross_spectra = []

# # Uncomment this block to calculate the spectra yourself
# # We use the mode coupling matrix `mc` from earlier
# for i in range(len(namap_list)):
#     for j in range(len(namap_list)):
#         if i >= j:
#             Cb = nw.compute_spectra(
#                 namap_list[i], namap_list[j], mc=mc)
#             if i > j:
#                 TT_cross_spectra += [Cb['TT']]

# Load the spectra from disk
TT_cross_spectra = np.loadtxt(path + 'TT_cross_spectra.txt')

## <font color='blue'> Exercise <font>

Try computing the spectra above instead of loading them from disk.

Now, build a new mask that does not include the point source mask this time. Re-compute the spectra using this new mask and plot the results below. How does the spectrum change when the point sources aren't masked?

(Ideally here we would also re-compute the mode coupling matrix using this new mask, but let's not do that here, as computing the mode coupling matrix takes a while.)

Now that we have our spectra we can calculate the mean standard error and the mean power spectrum.  We also want to undo any filttering that we did at the map processing stage by applying the corresponding transfer function to the power spectra.  In our case we had applied a Fourier (or k-space) filter when we used the `preprocess_fourier` function earlier on so we will now undo this kfilter by dividing it out.

In [ ]:
mean_Dltt = np.sum(TT_cross_spectra, axis=0) / len(TT_cross_spectra)
se_Dltt = np.std(TT_cross_spectra, axis=0) / np.sqrt(len(TT_cross_spectra))


# Now we undo k-space transfer function
k_func = np.loadtxt(path + 'deep56_kfilter_tfunc_sim.txt')

mean_Dltt = mean_Dltt/k_func[1:59,2]
se_Dltt = se_Dltt/k_func[1:59,2]

At this stage you could also calculate the mean TE or EE spectra if you had specified polarization components earlier when creating the namap_list.  

### Plotting our results

Now that we have a mean spectrum as well as some quick error bars we can plot the power spectrum.  Let's also read in a curve from CAMB in order to give us something to compare to by eye.  

In [ ]:
# Read in a camb fiducial spectrum for comparison
from pixell import powspec
camb_theory = powspec.read_spectrum(path + "camb_theory.dat")
cltt = camb_theory[0,0,:6000]
ls = np.arange(cltt.size)

# Set up our plot
fig, axes = plt.subplots(1,1, figsize=(10,6), sharex=True)

# plot our spectra
lb = mc.lb

plt.errorbar(lb, mean_Dltt,
                 fmt='C0.', 
                 yerr=(mean_Dltt / np.sqrt(2*lb+1) + se_Dltt),
                 lw=1, ms=4)
plt.plot(ls, cltt*ls**2./2/np.pi, lw=1, color='k', label = "fiducial")
plt.ylabel(r"$D_{\ell}$")
plt.legend()
plt.yscale('log')

If we take a moment to consider this power spectrum we can quite quickly note the differences between this and the one we produced using pixell in the previous notebook.  In that case we opted not to use a mode coupling matrix and instead used a much more simple approach that was quick to run but had a few glaring problems.

Most notably we see that by including the beam and by using the mode-coupling matrix we are able to recover power at intermediate ell values where previously we had been underestimating the fiducial spectrum. We also note that we do a better job at the higher $l$ values since we are using the correct point source mask. The TT spectrum at scales $l < 600$ are not used for the ACT cosmology analysis due to the mapmaking transfer function and other effects (see [Choi et al 2020](https://phy-act1.princeton.edu/public/saiola/act_dr4_C20.pdf) for more details). 

We take a moment to underline these improvements in order to show why these more computationally intensive methods are important for precision cosmology.  While the nawrapper routines are certainly slower than the quick example from the previous notebook, they also provide a much better estimate of the true underlying powerspectrum.  If you would like to explore these routines in more depth it's worth looking more into [nawrapper](https://github.com/xzackli/nawrapper) and trying to set up similar code to run on a cluster with a script instead of in notebook format.

### Analytic Covariance Matrices


Now let's take a moment to look at our approach to error analysis.  Above we took the very simple approach of using the standard error from the six split cross-spectra. That approach was pretty simple to apply and gave us a reasonable error bar to use for debugging purposes, however it isn't ideal.  

Here we take a moment to improve the approach by estimating the error analytically. To do a full error analysis we would want to include information from the simulations as well as systematic errors.  However, we can still improve on the standard error estimation above by including information from the four splits. 

By default, nawrapper will estimate the noise power spectrum using the difference in power between the auto- and cross-spectra between splits. In the DR4 pipeline, of course, these would be replaced by our full suite of noise simulations (see the next notebook).

In [ ]:
test = nw.nacov(namap_list[0], namap_list[1], mc, mc, mc)

In [ ]:
plt.plot(test.noise['T1T1'], label='noise')
plt.plot(test.signal['TT'], label='signal')
plt.legend()

Our aim here is to generate a mean covariance matrix that describes the covariance between TT spectra.  From there we can can get the analytic error bars by taking the square root of the diagonals of the covariance matrix.

Let's start with getting the mean covariance matrix.  We know that our average TT spectra is generated using 6 cross spectra, we can visualize what the various covariances corresponding to each set of spectra would be using the matrix example below.

$$\begin{bmatrix} TT_1TT_1 & TT_1TT_2 &...& TT_1TT_5 &TT_1TT_5\\ TT_2TT_1 & TT_2TT_2 &...& TT_2TT_5 &TT_2TT_6\\\vdots & \vdots & \ddots&\vdots &\vdots\\ TT_5TT_1 & TT_5TT_2 &...& TT_5TT_5 &TT_5TT_6 \\TT_6TT_1 & TT_6TT_2 &...& TT_6TT_5 &TT_6TT_6 \end{bmatrix} $$

In the above matrix each TT is one of 6 cross spectra and so each $TT_iTT_j$ corresponds to the covariance matrix between two TT spectra.  We can easily see that the matrix contains 36 covariance matrices, of which 6 correspond to auto covariance matrices and 30 correspond to cross covariance matrices.  However we are assuming that each split has the same covariance matrix which means that we will assume all 30 cross covariance matrices are the same and that all of the auto ones are also the same.  This means that we only need to calculate two of the matrices above in order to account for all 36.

Let's start by generating these to covariance matrices.  This process is also extremely computationally intensive, so we set the default behavior of the below cell to load the result from disk (but feel free to calculate it yourself):

In [ ]:
cross_cov = {}
auto_cov = {}

# # Uncomment this block to calculate the covariance matrices yourself
# cross_cov = nw.compute_covmat(namap_list[0], namap_list[1], bins, mc_11=mc, mc_12=mc, mc_22=mc)
# auto_cov = nw.compute_covmat(namap_list[0], namap_list[0], bins, mc_11=mc, mc_12=mc, mc_22=mc)

# Load the covariance matrices from disk
cross_cov['TTTT'] = np.loadtxt(path + 'cross_cov_TTTT.txt')
auto_cov['TTTT'] = np.loadtxt(path + 'auto_cov_TTTT.txt')

In the above example we are only using the Temperature maps so we only have the `TTTT` components, however we could also have `TETT` or `EEEE` etc. components if we were including polarization spectra as well.

Now that we have the two matrices we need to combine them to get the mean covariance matrix.  In essence we want to weight the two matrices and then just add them.  In practice this comes down to weighing them in accordance with how often they are used in the analysis.  

If we refer back to our matrix from earlier we can basically count the number of auto covariance matrices and cross covariance matrices. Since we are assuming that all of the cross covariance matrices are the same we will simply weigh the cross covariance matrix by 30 and similarly we weigh the auto covariance matrix by 6.  We then normalize by the number of spectra.

Instead of plugging in the numbers we'll take a more mathematical approach here so that you could adjust it if you had a different number of spectra.  

In [ ]:
from scipy.special import comb

# compute the number of TT cross-spectra from the number of splits
nsplits = 4
n_spec = comb(nsplits, 2)

# weight auto and cross covmats to get mean covmat
cross_weight = n_spec**2 - n_spec
auto_weight = n_spec
combined_TT_cov = (cross_cov['TTTT']*cross_weight/n_spec + auto_cov['TTTT']*auto_weight/n_spec)/(n_spec**2)

Now that we have our covariance matrix. getting the new error bars is simply a matter of taking the square root of the variance where the variance is given by the diagonal of the covariance matrix. 

For convenience, we also plot one of the spectra from the D56 region presented in Choi et. al., along with its errorbars. As discussed in the text, we use a different code than `nawrapper`, along with simulations to estimate noise and bin covariance, so the simplified notebook treatment will not match the DR4 results exactly.  However, as we can see in the plot below `nawrapper` reproduces our results extremely well.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,6), sharex=True)

# load DR4 spectrum
ell, d_ell_dr4, sig_d_ell_dr4 = np.loadtxt(path + 'deep56_TT_spectrum.txt').T

# plot our spectra
lb = mc.lb

ax.errorbar(lb, mean_Dltt,
            fmt='k.', 
            yerr=(mean_Dltt / np.sqrt(2 * lb + 1) + se_Dltt),
            lw=1, ms=3, label="Nawrapper Spectrum with Standard error")
ax.errorbar(lb + 4, mean_Dltt, # we slightly offset to not overlap
            fmt='r.', 
            yerr=np.sqrt(np.diag(combined_TT_cov)) / (lb * (lb+1) / 2 / np.pi),
            lw=1, ms=3, label="Nawrapper Spectrum with Analytic error")
ax.errorbar(ell + 8, d_ell_dr4, fmt = 'b.', yerr = sig_d_ell_dr4, lw = 1, ms = 3, label = 'DR4 Spectrum with DR4 error')

ax.set_xlabel('$\ell$', fontsize = 16)
ax.set_ylabel(r"$D_{\ell}$", fontsize = 16)
ax.legend(frameon=True, fontsize = 14)
plt.title("Comparing Nawrapper to our DR4 Spectrum for this Patch", fontsize = 14)
plt.yscale('log')
plt.ylim(.1, 1e4)
plt.tight_layout()

Note that the errors here do not include any corrections for systematic errors due to the window function, calibration, or mapmaking transfer function uncertainty. In [Choi et al 2020](https://phy-act1.princeton.edu/public/saiola/act_dr4_C20.pdf) these corrections are applied after coadding the power spectra over seasons and arrays.

## Continue to next section

[Section 9](Section_9_Noise_sims.ipynb): Generating Noise Simulations 